# ***Movie Recommendation System using Cosine Similarity***

# *IMPORING LIBRARIES AND CSV FILE*

In [ ]:
#importing the libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Importing the dataset
df = pd.read_csv("/content/drive/MyDrive/movie_dataset.csv")

# *DATA INSPECTION*

In [ ]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
#to check wheateher dataset has null values or not
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [ ]:
df.describe(include = 'all')

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
count,4803.000000,4.803000e+03,4775,1712,4803.000000,4391,4803,4803,4800,4803.000000,...,4801.000000,4803,4803,3959,4803,4803.000000,4803.000000,4760,4803,4773
unique,NaN,NaN,1168,1691,NaN,4219,37,4801,4800,NaN,...,NaN,544,3,3944,4800,NaN,NaN,4741,4776,2349
top,NaN,NaN,Drama,http://www.missionimpossible.com/,NaN,independent film,en,Out of the Blue,"In the 22nd century, a paraplegic Marine is di...",NaN,...,NaN,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Based on a true story.,The Host,NaN,NaN,William Shatner Leonard Nimoy DeForest Kelley ...,[],Steven Spielberg
freq,NaN,NaN,370,4,NaN,55,4505,2,1,NaN,...,NaN,3171,4795,3,2,NaN,NaN,6,28,27
mean,2401.000000,2.904504e+07,NaN,NaN,57165.484281,NaN,NaN,NaN,NaN,21.492301,...,106.875859,NaN,NaN,NaN,NaN,6.092172,690.217989,NaN,NaN,NaN
std,1386.651002,4.072239e+07,NaN,NaN,88694.614033,NaN,NaN,NaN,NaN,31.816650,...,22.611935,NaN,NaN,NaN,NaN,1.194612,1234.585891,NaN,NaN,NaN
min,0.000000,0.000000e+00,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,0.000000,...,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,1200.500000,7.900000e+05,NaN,NaN,9014.500000,NaN,NaN,NaN,NaN,4.668070,...,94.000000,NaN,NaN,NaN,NaN,5.600000,54.000000,NaN,NaN,NaN
50%,2401.000000,1.500000e+07,NaN,NaN,14629.000000,NaN,NaN,NaN,NaN,12.921594,...,103.000000,NaN,NaN,NaN,NaN,6.200000,235.000000,NaN,NaN,NaN
75%,3601.500000,4.000000e+07,NaN,NaN,58610.500000,NaN,NaN,NaN,NaN,28.313505,...,118.000000,NaN,NaN,NaN,NaN,6.800000,737.000000,NaN,NaN,NaN


In [ ]:
#Checking Duplicates
len(df[df.duplicated()])

0

In [ ]:
#checking Columns
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [ ]:
#use the features most relevant to us.
features = ['keywords','cast','genres','director']

#data preprocessing and replace any rows having NaN values with a empty string, so it does not generate an error while running the code
for feature in features:
    df[feature] = df[feature].fillna('')

In [ ]:
#define a function which combine all relevant feature to single string
def combined_features(row):
    return (row['keywords']+" "+row['cast']+" "+row['genres']+" "+row['director'])

#We add new column to our existing dataframe and apply to each row with axis=1
df["combined_features"] = df.apply(combined_features, axis =1)
df.head(2)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,combined_features
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron,culture clash future space war space colony so...
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski,ocean drug abuse exotic island east india trad...


In [ ]:
df['combined_features']

0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6 Dan...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
                              ...                        
4798    united states\u2013mexico barrier legs arms pa...
4799     Edward Burns Kerry Bish\u00e9 Marsha Dietlein...
4800    date love at first sight narration investigati...
4801     Daniel Henney Eliza Coupe Bill Paxton Alan Ru...
4802    obsession camcorder crush dream girl Drew Barr...
Name: combined_features, Length: 4803, dtype: object

# *Extracting Features*

In [ ]:
#use CountVectorizer’s fit.tranform to count the number of texts and we will print the transformed matrix count_matrix into an array
cv = CountVectorizer()
count_matrix = cv.fit_transform(df["combined_features"])
print("Count Matrix:", count_matrix.toarray())

Count Matrix: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
#Using the cosine similarity from sklearn to compute similarity between two movies
#o/p ranges --> 0-1 where 0 -> No similarity and 1 -> 100% Similar
cosine_sim = cosine_similarity(count_matrix) 

In [ ]:
cosine_sim

array([[1.        , 0.10540926, 0.12038585, ..., 0.        , 0.        ,
        0.        ],
       [0.10540926, 1.        , 0.0761387 , ..., 0.03651484, 0.        ,
        0.        ],
       [0.12038585, 0.0761387 , 1.        , ..., 0.        , 0.11145564,
        0.        ],
       ...,
       [0.        , 0.03651484, 0.        , ..., 1.        , 0.        ,
        0.04264014],
       [0.        , 0.        , 0.11145564, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04264014, 0.        ,
        1.        ]])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
cosine_sim_ = pd.DataFrame(cosine_sim)
cosine_sim_


,0,1,2,3,4,5,6,7,8,9,...,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.000000,0.105409,0.120386,0.037743,0.230940,0.192450,0.0,0.140546,0.082061,0.117851,...,0.000000,0.000000,0.000000,0.070273,0.000000,0.075485,0.000000,0.000000,0.000000,0.00000
1,0.105409,1.000000,0.076139,0.035806,0.073030,0.255604,0.0,0.066667,0.116775,0.111803,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.035806,0.000000,0.036515,0.000000,0.00000
2,0.120386,0.076139,1.000000,0.163572,0.208514,0.125109,0.0,0.190347,0.088911,0.170251,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.081786,0.000000,0.000000,0.111456,0.00000
3,0.037743,0.035806,0.163572,1.000000,0.039223,0.078446,0.0,0.035806,0.083624,0.120096,...,0.089984,0.050637,0.125436,0.107417,0.087706,0.153846,0.000000,0.039223,0.000000,0.00000
4,0.230940,0.073030,0.208514,0.039223,1.000000,0.200000,0.0,0.219089,0.042640,0.163299,...,0.000000,0.051640,0.000000,0.109545,0.000000,0.039223,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,0.075485,0.035806,0.081786,0.153846,0.039223,0.039223,0.0,0.071611,0.000000,0.040032,...,0.000000,0.050637,0.041812,0.035806,0.087706,1.000000,0.000000,0.000000,0.000000,0.00000
4799,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.059235,0.000000,0.000000,0.000000,0.000000,1.000000,0.091766,0.000000,0.00000
4800,0.000000,0.036515,0.000000,0.039223,0.000000,0.040000,0.0,0.000000,0.000000,0.000000,...,0.045883,0.051640,0.042640,0.036515,0.000000,0.000000,0.091766,1.000000,0.000000,0.04264
4801,0.000000,0.000000,0.111456,0.000000,0.000000,0.000000,0.0,0.000000,0.113961,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000


In [ ]:
df['title']

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                                 El Mariachi
4799                                   Newlyweds
4800                   Signed, Sealed, Delivered
4801                            Shanghai Calling
4802                           My Date with Drew
Name: title, Length: 4803, dtype: object

In [ ]:
#The next step is to take as input a movie that the user likes in the movie_user_likes variable. We are gonna use content based filtering
movie_user_likes = (input("Enter the movie that user like to watch : "))

#function to get the index from the name of this movie. The index will be saved in the movie_index variable.
def get_index_from_title(title):
    return df[df.title == title]["index"].values[0]
  
movie_index = get_index_from_title(movie_user_likes)
print('movie index is :',movie_index)

Enter the movie that user like to watch : The Dark Knight Rises
movie index is : 3


In [ ]:
#generate a list of similar movies
similar_movies = list(enumerate(cosine_sim[movie_index]))
similar_movies

[(0, 0.03774256780481986),
 (1, 0.03580574370197164),
 (2, 0.16357216402190616),
 (3, 0.9999999999999997),
 (4, 0.03922322702763681),
 (5, 0.07844645405527362),
 (6, 0.0),
 (7, 0.03580574370197164),
 (8, 0.08362420100070908),
 (9, 0.12009611535381537),
 (10, 0.11766968108291043),
 (11, 0.23533936216582085),
 (12, 0.04089304100547654),
 (13, 0.03922322702763681),
 (14, 0.14824986333222023),
 (15, 0.0),
 (16, 0.03522349768381735),
 (17, 0.04181210050035454),
 (18, 0.07161148740394328),
 (19, 0.04181210050035454),
 (20, 0.04003203845127179),
 (21, 0.04003203845127179),
 (22, 0.0),
 (23, 0.0),
 (24, 0.06933752452815364),
 (25, 0.08770580193070293),
 (26, 0.03706246583305506),
 (27, 0.08006407690254358),
 (28, 0.08006407690254358),
 (29, 0.1254363015010636),
 (30, 0.07692307692307693),
 (31, 0.07844645405527362),
 (32, 0.07412493166611012),
 (33, 0.07412493166611012),
 (34, 0.0),
 (35, 0.08559209850218259),
 (36, 0.08006407690254358),
 (37, 0.0),
 (38, 0.038461538461538464),
 (39, 0.1153846

In [ ]:
#sort the movies in the list similar_movies
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)
sorted_similar_movies

[(3, 0.9999999999999997),
 (119, 0.7307692307692306),
 (65, 0.6923076923076922),
 (4638, 0.45291081365783836),
 (2793, 0.39467610868816316),
 (1196, 0.37630890450319093),
 (428, 0.3602883460614461),
 (1359, 0.3530090432487313),
 (210, 0.3396831102433788),
 (1720, 0.3335621924974955),
 (2371, 0.3335621924974955),
 (1740, 0.32776066832815337),
 (3332, 0.3149448894660932),
 (3359, 0.29417420270727607),
 (3819, 0.2926847035024818),
 (3966, 0.2926847035024818),
 (4099, 0.29268470350248177),
 (2790, 0.29134281629169184),
 (1013, 0.28625128703833574),
 (1792, 0.28625128703833574),
 (2398, 0.28625128703833574),
 (1503, 0.2853908964926965),
 (3293, 0.28022426915890253),
 (1986, 0.27735009811261463),
 (4664, 0.27735009811261463),
 (1234, 0.27456258919345766),
 (3073, 0.2692307692307693),
 (95, 0.264197974633739),
 (813, 0.264197974633739),
 (1664, 0.264197974633739),
 (968, 0.2631174057921088),
 (1674, 0.2631174057921088),
 (1727, 0.2631174057921088),
 (1803, 0.2631174057921088),
 (3637, 0.26311

In [ ]:
#define a simple function that takes the index number and covert it into the movie title as in the dataframe.
def get_title_from_index(index):
    return df[df.index == index]["title"].values[0]

In [ ]:
# print the 15 most similar movies from a pool of 4802 movies.
i=0
for movie in sorted_similar_movies:
    print(get_title_from_index(movie[0]))
    i=i+1
    if i>15:
        break

The Dark Knight Rises
Batman Begins
The Dark Knight
Amidst the Devil's Wings
The Killer Inside Me
The Prestige
Batman Returns
Batman
Batman & Robin
Kick-Ass
RockNRolla
Kick-Ass 2
Harry Brown
In Too Deep
Defendor
Point Blank
